Lybraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score,ConfusionMatrixDisplay,confusion_matrix,precision_score,recall_score,roc_curve,roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from tensorflow import keras
import datetime

### EDA

In [ ]:
train_path='data/test_motion_data.csv'
test_path='data/train_motion_data.csv'

#### Train data

In [ ]:
train_data=pd.read_csv(train_path)
train_data.head(1)

In [ ]:
train_data.describe

In [ ]:
train_data.isna().sum()

In [ ]:
train_data=train_data.sort_values(by='Timestamp')

In [ ]:
train_data.Timestamp.unique().shape

There are roughly 2 measurements per second

In [ ]:
train_data.info()

In [ ]:
train_data.Timestamp.value_counts().describe()

It proves that there is not a single timestamp that is replicated many times.

In [ ]:
train_data.columns

In [ ]:
train_data['Timestamp']=train_data['Timestamp'].astype('int')

train_data['Class']=train_data['Class'].astype('category')
train_data['class_code']=train_data['Class'].cat.codes

In [ ]:
var_columns=['AccX','AccY','AccZ','GyroX','GyroY','GyroZ']
label_columns=['class_code','Class']

In [ ]:
index=(train_data.Timestamp.value_counts()>1).index
measument1=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=measument1-measument2
fig,axes=plt.subplots(1,len(var_columns),figsize=(10,6))
for column,ax in zip(var_columns,axes.flatten()):    
    sns.boxplot(data=diff,y=column,ax=ax)
plt.tight_layout()

There are a lot of outliers, e.i., There is many measuments in the same timestamp that are desagreeing badly. However, it might be due to is amplitude. Then lets take a look in the boxplot normalized. 

In [ ]:
index=(train_data.Timestamp.value_counts()>1).index
measument1=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=(measument1-measument2)/0.5*(measument2+measument1)
fig,axes=plt.subplots(1,len(var_columns),figsize=(10,6))
for column,ax in zip(var_columns,axes.flatten()):    
    sns.boxplot(data=diff,y=column,ax=ax)
plt.tight_layout()

The gyroscope data is not desagreeing much. But, the accelaration that It massive desagreement in some measuments. 
 


In [ ]:
index=(train_data.Timestamp.value_counts()>1).index
measument1=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=(measument1-measument2)/0.5*(measument2+measument1)
diff=diff.reset_index().merge(train_data.drop_duplicates('Timestamp')[['Timestamp','Class']],right_on='Timestamp',left_on='index')
used_columns=[column for column in var_columns if 'Acc' in column]
fig,axes=plt.subplots(1,len(used_columns),figsize=(10,6))
for column,ax in zip(used_columns,axes.flatten()):
    sns.boxplot(data=diff,y=column,ax=ax,x='Class')
plt.tight_layout()

level of agreeament

[1]Bland, J. Martin, and DouglasG Altman. "Statistical methods for assessing agreement between two methods of clinical measurement." The lancet 327.8476 (1986): 307-310.


In [ ]:
index=(train_data.Timestamp.value_counts()>1).index
measument1=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=measument1-measument2
mean=(measument1+measument2)*0.5
used_columns=[column for column in var_columns if 'Acc' in column]
fig,axes=plt.subplots(len(used_columns),1,figsize=(10,6),sharex=True)
for column,ax in zip(used_columns,axes.flatten()):    
    ax.scatter(x=mean[column],y=diff[column])
    ax.hlines(np.mean(diff[column])+2*np.std(diff[column]),mean[used_columns].min().min(),mean[used_columns].max().max(),linestyles='dashed',color='black')
    ax.hlines(np.mean(diff[column])-2*np.std(diff[column]),mean[used_columns].min().min(),mean[used_columns].max().max(),linestyles='dashed',color='black')
plt.tight_layout()


Even though, the data are quite odd and show significant different, Using [1] one might argui that both data agree.

Also, With this it seems that there is not a specific range where the data desagree more.

Since both measures may have differnt signs I will be taken the mean of its module, I wonder if there is some behaviour hidden due it. 

In [ ]:
index=(train_data.Timestamp.value_counts()>1).index
measument1=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=train_data.set_index(train_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=measument1-measument2
mean=(np.abs(measument1)+np.abs(measument2))*0.5
used_columns=[column for column in var_columns if 'Acc' in column]
fig,axes=plt.subplots(len(used_columns),1,figsize=(10,6),sharex=True)
for column,ax in zip(used_columns,axes.flatten()):    
    ax.scatter(x=mean[column],y=diff[column])

plt.tight_layout()


Adding new features.

In [ ]:
train_data['Acc_mod']=0.5*(train_data['AccX']**2+train_data['AccY']**2+train_data['AccZ']**2)
var_columns.append('Acc_mod')

Dataset balance

In [ ]:
train_data['Class'].value_counts(normalize=True).plot(kind='bar')

Data aumentation

In [ ]:
missing_seconds=set(train_data['Timestamp'].unique()).difference(set(range(train_data['Timestamp'].min(),train_data['Timestamp'].max()+1)))
missing_seconds

With this we ensure the signal continuity

Bellow, it has been created some code for fixing some possible descontinuity

In [ ]:
avaible_seconds=train_data.Timestamp.unique()

for second in missing_seconds:
    new_index=len(train_data)+1
    train_data.loc[new_index,['Timestamp']]=[second]
    train_data.loc[new_index,var_columns]=[np.nan]*len(var_columns)
   
    for offset in range(1,11):
        filled=False
        if second-offset in avaible_seconds:
            train_data.loc[new_index,label_columns]=train_data.drop_duplicates('Timestamp').set_index('Timestamp').loc[second-offset,label_columns]
            filled=True
            break
        elif second+offset in avaible_seconds:
            train_data.loc[new_index,label_columns]=train_data.drop_duplicates('Timestamp').set_index('Timestamp').loc[second+offset,label_columns]
            filled=True
            break

    if not filled: print('Method did not work')
    train_data.loc[new_index,label_columns]=[np.nan]*len(label_columns)
    
train_data=train_data.sort_values('Timestamp').reset_index(drop=True)
train_data[var_columns]=train_data[var_columns].interpolate(method='polynomial',order=5)[var_columns]=train_data[var_columns].interpolate(method='linear')

Observing the data as a signal

In [ ]:
train_data['Class'].map({'SLOW':'green','NORMAL':'blue','AGGRESSIVE':'red'}).value_counts()

In [ ]:
fig,axes=plt.subplots(len(var_columns),1,figsize=(8,3*len(var_columns)))

for ax,column in zip (axes.flatten(),var_columns):
    grouped=train_data.groupby('Class')
    color={'SLOW':'green','NORMAL':'blue','AGGRESSIVE':'red'}
    for key,group in grouped:
        group.plot(ax=ax,x='Timestamp',y=column,label=key,color=color[key])
    ax.set_title(column)
plt.tight_layout()

FFT

In [ ]:
def fourier_analysis(data,fft_ax,psd_ax,prefix=''):
    fft=np.fft.fft(data)
    freq=np.fft.fftfreq(len(data),d=1)

    fft_ax.plot(freq,fft.real,color='red',alpha=0.5,label='real')
    fft_ax.plot(freq,fft.imag,color='blue',alpha=0.6,label='imag')
    fft_ax.set_xlim(left=0)
    fft_ax.set_title(prefix+' FFT')
    fft_ax.legend()
    psd=(1/len(data))*(fft.real**2+fft.imag**2)
    psd_ax.set_title(prefix+' PSD')
    psd_ax.plot(freq,psd,color='blue',alpha=0.7)
    psd_ax.set_xlim(left=-0.01)
    

In [ ]:
fig,axes=plt.subplots(len(var_columns),2,figsize=(8,3*len(var_columns)))

for ax,column in zip (axes,var_columns):
    fourier_analysis(train_data[column],ax[0],ax[1],prefix=column)
plt.tight_layout()

Looking the fft of each class, since showed before, there are three continuos periods in data

In [ ]:
fig,axes=plt.subplots(len(var_columns),6,figsize=(16,3*len(var_columns)))

for ax,column in zip (axes,var_columns):
    grouped=train_data.groupby('Class')
    for groupd_index,(key,group) in enumerate(grouped):
        data=grouped.get_group(key)[column]
        fourier_analysis(data,ax[groupd_index*2],ax[groupd_index*2+1],prefix=f'{key} - {column}')
plt.tight_layout()

spectogram

In [ ]:
def spectogram_analysis(data,ax,prefix=''):
    f, t, Sxx = signal.spectrogram(data, 1)
    im=ax.pcolormesh(t, f, Sxx, shading='gouraud')
    ax.set_ylabel('Frequency [Hz]')
    ax.set_xlabel('Time [sec]')
    ax.set_title(prefix)
    plt.colorbar(im,ax=ax)

In [ ]:
fig,axes=plt.subplots(len(var_columns),4,figsize=(16,2*len(var_columns)))

for ax,column in zip (axes,var_columns):
    data=train_data[column]
    spectogram_analysis(data,ax[0],f'All classes - {column}')
    grouped=train_data.groupby('Class')
    for groupd_index,(key,group) in enumerate(grouped):
        data=group[column]
        spectogram_analysis(data,ax[groupd_index+1],prefix=f'{key} - {column}')
plt.tight_layout()

#### Test data - The previus analysys was replicated

In [ ]:
test_data=pd.read_csv(test_path)
test_data.head(1)

In [ ]:
test_data.describe()

In [ ]:
test_data.isna().sum()

In [ ]:
test_data=test_data.sort_values(by='Timestamp')

In [ ]:
test_data.Timestamp.unique().shape

There are roughly 2 measurements per second

In [ ]:
test_data.info()

In [ ]:
test_data.Timestamp.value_counts().describe()

It proves that there is not a single timestamp that is replicated many times.

In [ ]:
test_data.columns

In [ ]:
test_data['Timestamp']=test_data['Timestamp'].astype('int')

test_data['Class']=test_data['Class'].astype('category')
test_data['class_code']=test_data['Class'].cat.codes

In [ ]:
var_columns=['AccX','AccY','AccZ','GyroX','GyroY','GyroZ']
label_columns=['class_code','Class']

In [ ]:
index=(test_data.Timestamp.value_counts()>1).index
measument1=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=measument1-measument2
fig,axes=plt.subplots(1,len(var_columns),figsize=(10,6))
for column,ax in zip(var_columns,axes.flatten()):    
    sns.boxplot(data=diff,y=column,ax=ax)
plt.tight_layout()

In [ ]:
index=(test_data.Timestamp.value_counts()>1).index
measument1=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=(measument1-measument2)/0.5*(measument2+measument1)
fig,axes=plt.subplots(1,len(var_columns),figsize=(10,6))
for column,ax in zip(var_columns,axes.flatten()):    
    sns.boxplot(data=diff,y=column,ax=ax)
plt.tight_layout()

In [ ]:
index=(test_data.Timestamp.value_counts()>1).index
measument1=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=(measument1-measument2)/0.5*(measument2+measument1)
diff=diff.reset_index().merge(test_data.drop_duplicates('Timestamp')[['Timestamp','Class']],right_on='Timestamp',left_on='index')
used_columns=[column for column in var_columns if 'Acc' in column]
fig,axes=plt.subplots(1,len(used_columns),figsize=(10,6))
for column,ax in zip(used_columns,axes.flatten()):
    sns.boxplot(data=diff,y=column,ax=ax,x='Class')
plt.tight_layout()

level of agreeament

[1]Bland, J. Martin, and DouglasG Altman. "Statistical methods for assessing agreement between two methods of clinical measurement." The lancet 327.8476 (1986): 307-310.


In [ ]:
index=(test_data.Timestamp.value_counts()>1).index
measument1=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=measument1-measument2
mean=(measument1+measument2)*0.5
used_columns=[column for column in var_columns if 'Acc' in column]
fig,axes=plt.subplots(len(used_columns),1,figsize=(10,6),sharex=True)
for column,ax in zip(used_columns,axes.flatten()):    
    ax.scatter(x=mean[column],y=diff[column])
    ax.hlines(np.mean(diff[column])+2*np.std(diff[column]),mean[used_columns].min().min(),mean[used_columns].max().max(),linestyles='dashed',color='black')
    ax.hlines(np.mean(diff[column])-2*np.std(diff[column]),mean[used_columns].min().min(),mean[used_columns].max().max(),linestyles='dashed',color='black')
plt.tight_layout()


In [ ]:
index=(test_data.Timestamp.value_counts()>1).index
measument1=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='first',subset='Timestamp').loc[index,var_columns]
measument2=test_data.set_index(test_data.Timestamp).drop_duplicates(keep='last',subset='Timestamp').loc[index,var_columns]
diff=measument1-measument2
mean=(np.abs(measument1)+np.abs(measument2))*0.5
used_columns=[column for column in var_columns if 'Acc' in column]
fig,axes=plt.subplots(len(used_columns),1,figsize=(10,6),sharex=True)
for column,ax in zip(used_columns,axes.flatten()):    
    ax.scatter(x=mean[column],y=diff[column])

plt.tight_layout()


Adding new features.

In [ ]:
test_data['Acc_mod']=0.5*(test_data['AccX']**2+test_data['AccY']**2+test_data['AccZ']**2)
var_columns.append('Acc_mod')

Dataset balance

In [ ]:
test_data['Class'].value_counts(normalize=True).plot(kind='bar')

Data aumentation

In [ ]:
missing_seconds=set(test_data['Timestamp'].unique()).difference(set(range(test_data['Timestamp'].min(),test_data['Timestamp'].max()+1)))
missing_seconds

With this we ensure the signal continuity

Observing the data as a signal

In [ ]:
fig,axes=plt.subplots(len(var_columns),1,figsize=(8,3*len(var_columns)))

for ax,column in zip (axes.flatten(),var_columns):
    grouped=test_data.groupby('Class')
    color={'SLOW':'green','NORMAL':'blue','AGGRESSIVE':'red'}
    for key,group in grouped:
        group.plot(ax=ax,x='Timestamp',y=column,label=key,color=color[key])
    ax.set_title(column)
plt.tight_layout()

FFT

In [ ]:
fig,axes=plt.subplots(len(var_columns),2,figsize=(8,3*len(var_columns)))

for ax,column in zip (axes,var_columns):
    fourier_analysis(test_data[column],ax[0],ax[1],prefix=column)
plt.tight_layout()

Looking the fft of each class, since showed before, there are three continuos periods in data

In [ ]:
fig,axes=plt.subplots(len(var_columns),6,figsize=(16,3*len(var_columns)))

for ax,column in zip (axes,var_columns):
    grouped=test_data.groupby('Class')
    for groupd_index,(key,group) in enumerate(grouped):
        data=grouped.get_group(key)[column]
        fourier_analysis(data,ax[groupd_index*2],ax[groupd_index*2+1],prefix=f'{key} - {column}')
plt.tight_layout()

spectogram

In [ ]:
def spectogram_analysis(data,ax,prefix=''):
    f, t, Sxx = signal.spectrogram(data, 1)
    im=ax.pcolormesh(t, f, Sxx, shading='gouraud')
    ax.set_ylabel('Frequency [Hz]')
    ax.set_xlabel('Time [sec]')
    ax.set_title(prefix)
    plt.colorbar(im,ax=ax)

In [ ]:
fig,axes=plt.subplots(len(var_columns),4,figsize=(16,2*len(var_columns)))

for ax,column in zip (axes,var_columns):
    data=test_data[column]
    spectogram_analysis(data,ax[0],f'All classes - {column}')
    grouped=test_data.groupby('Class')
    for groupd_index,(key,group) in enumerate(grouped):
        data=group[column]
        spectogram_analysis(data,ax[groupd_index+1],prefix=f'{key} - {column}')
plt.tight_layout()

### Data processing to aplly AI

The models that will built aims to find which data are related to agressive driving, then, it will became a binary classification problem.

The input data should be a sequence of seconds, since the acceleration of a given seconds does not provide enough information about the driving beahaviour

In [ ]:
test_data[label_columns].value_counts()


In [ ]:
train_data[label_columns].value_counts()


In [ ]:
map={0:1,2:0,1:0}
train_data.class_code=train_data.class_code.map(map)
test_data.class_code=test_data.class_code.map(map)

In [ ]:
train_data['class_code'].value_counts()


In [ ]:
test_data['class_code'].value_counts()


In [ ]:
def create_sequences(sequence_size,class_column,classes_key:list,df):
    X=[]
    Y=[]
    for key in classes_key:
        
        train_class=df[df[class_column]==key]    
        size=len(train_class)//sequence_size*sequence_size
        X_temp=train_class[var_columns.copy()].values[:size].reshape(-1,sequence_size,len(var_columns))
        Y_temp=np.array([key]*len(X_temp)).reshape(-1,1)
        if len(X)!=0:
            X=np.vstack((X,X_temp))
            Y=np.vstack((Y,Y_temp))
        else:
            X=X_temp
            Y=Y_temp
    return X,Y

In [ ]:
X_train_sequece,Y_train_sequence=create_sequences(3,'class_code',[0,1,2],train_data)
X_test_sequence,Y_test_sequence=create_sequences(3,'class_code',[0,1,2],test_data)

In [ ]:
np.random.seed(123)
X_test_sequence,X_val_sequence,Y_test_sequence,Y_val_sequence=train_test_split(X_test_sequence,Y_test_sequence,test_size=0.5,stratify=Y_test_sequence)

In [ ]:
X_train=train_data[var_columns]
Y_train=train_data['class_code']
X_test=test_data[var_columns]
Y_test=test_data['class_code']
X_test,X_val,Y_test,Y_val=train_test_split(X_test,Y_test,test_size=0.5,stratify=Y_test)


In [ ]:
sum(Y_train==1)

### Models

In [ ]:
class ClassificationModel():
    def __init__(self,model,Xtrain,Ytrain,scaler=False,random_state=123,**kwargs):
        self.random_state=random_state
        if scaler:
            self.model=make_pipeline(StandardScaler(),model(**kwargs))
        else:
            self.model=model(**kwargs)
        self.Xtrain=Xtrain
        self.Ytrain=Ytrain
        self._get_label_proportion(Ytrain,prefix='train')
        
    
    def fit(self):      
        np.random.seed(self.random_state)  
        self.model.fit(self.Xtrain,self.Ytrain)

    def evaluate(self,Xtest,Ytest):
        self._get_label_proportion(Ytest,prefix='test')
        self.pred=self.model.predict(X_test)
        self._get_accuracy(Ytest)
        self._get_confusion_matrix(Ytest)
        self._get_class_report(Ytest)

    def _get_confusion_matrix(self,Ytest):
        cm = confusion_matrix(Ytest,self.pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
    
    def _get_class_report(self,Ytest):
        report=classification_report(Ytest,self.pred)
        print(report)

    def _get_accuracy(self,Ytest):
        accuracy=accuracy_score(Ytest,self.pred)
        print(f'The accuracy observed was {accuracy*100:.0f} %')
    
    def _get_label_proportion(self,Y,prefix=''):
        true_rate=sum(Y==1)/len(Y)
        print(f'{prefix}: The dataset has {true_rate*100:.0f}% of positive')
    

### Machine learning

Logistic Regression

In [ ]:
LR=ClassificationModel(LogisticRegression,X_train,Y_train,n_jobs=-1)
LR.fit()
LR.evaluate(X_val,Y_val)

Scaled

In [ ]:
LR=ClassificationModel(LogisticRegression,X_train,Y_train,scaler=True)
LR.fit()
LR.evaluate(X_val,Y_val)

The dummiest classifier would guess everything negative and would have the same result.

SVM

In [ ]:
SVC_model=ClassificationModel(SVC,X_train,Y_train,scaler=True)
SVC_model.fit()
SVC_model.evaluate(X_val,Y_val)

Random Forest

In [ ]:
RF=ClassificationModel(RandomForestClassifier,X_train,Y_train,n_jobs=-1)
RF.fit()
RF.evaluate(X_val,Y_val)

Naive Bayes

In [ ]:
MB=ClassificationModel(GaussianNB,X_train,Y_train)
MB.fit()
MB.evaluate(X_val,Y_val)

Xgboost

In [ ]:
XGB=ClassificationModel(XGBClassifier,X_train,Y_train)
XGB.fit()
XGB.evaluate(X_val,Y_val)

None of the models have performed well, as it was mentioned before, our base line shloud be ~69%.

### Deep Learning

MLP

In [ ]:
from abc import abstractclassmethod
class DLModel():

    def __init__(self,Xtrain,Ytrain,Xval,Yval,Xtest,Ytest):
        self.Xtrain=Xtrain
        self.Ytrain=Ytrain
        self.Xval=Xval
        self.Yval=Yval
        self.Xtest=Xtest
        self.Ytest=Ytest
        self.training_history=[]
        self.model=self.model_compiler()
    
    @property
    def architecture(self):
        return self.model.summary()
    
    @abstractclassmethod
    def model_compiler(self):
        pass

    @abstractclassmethod
    def train_params(self)->dict:
        pass   

    def train(self,seed=123,verbose=0):
        np.random.seed(seed)
        history=self.model.fit(self.Xtrain,self.Ytrain,validation_data=(self.Xval,self.Yval),verbose=verbose,**self.training_params())
        self.training_history.append(history)     

    def test(self):
        threshold=0.5
        self.predictions=self.model.predict(self.Xtest)
        self.binary_predictions=(self.predictions>threshold).astype(int)
        test_accuracy=accuracy_score(self.Ytest,self.binary_predictions)        
        test_precision=precision_score(self.Ytest,self.binary_predictions)
        test_recall=recall_score(self.Ytest,self.binary_predictions)
        auc=roc_auc_score(self.Ytest,self.binary_predictions)
        print(f'Test accuracy : {test_accuracy*100:.0f} %')
        print(f'Test precision : {test_precision*100:.0f} %')
        print(f'Test recall : {test_recall*100:.0f} %')
        print(f'Model AUC : {auc*100:.0f} %')
        self.confusion_matrix_plot()


    @staticmethod
    def __unpack_data_history(history_record,key):
        key=DLModel.__get_metric_key_by_prefix(key,history_record)
        metric=[]
        for train_data in history_record:
            metric.extend(train_data.history[key])
        
        return metric
    
    @staticmethod
    def __get_metric_key_by_prefix(key,history_record):
        key_in_history=''
        if 'val' in key:
            for metric in history_record[-1].history.keys():
                if key in metric:
                    key_in_history=metric
                    break
        else:
            for metric in history_record[-1].history.keys():
                if key in metric and 'val' not in metric:
                    key_in_history=metric
                    break

        return key_in_history

    def loss_plot(self,ax=None):
        if not ax:
            loss=DLModel.__unpack_data_history(self.training_history,'loss')
            plt.title('loss')
            plt.plot(loss)
            plt.xlabel('epochs')
            plt.show()
        else:
            pass

    def accuracy_plot(self,ax=None):
        if not ax:        
            accuracy=DLModel.__unpack_data_history(self.training_history,'binary_accuracy')
            val_accuracy=DLModel.__unpack_data_history(self.training_history,'val_binary_accuracy')
            plt.title('Accuracy')
            plt.plot(accuracy,label='train')
            plt.plot(val_accuracy,label='val')
            plt.legend()
            plt.xlabel('epochs')
            plt.grid(True)
            plt.show()
        else:
            pass


    def precision_recall_plot(self,ax=None):
        if not ax:
            precision=DLModel.__unpack_data_history(self.training_history,'precision')
            recall=DLModel.__unpack_data_history(self.training_history,'recall')
            val_precision=DLModel.__unpack_data_history(self.training_history,'val_precision')
            val_recall=DLModel.__unpack_data_history(self.training_history,'val_recall')
            plt.subplot(1,2,1)
            plt.title('Precision')
            plt.plot(precision,label='Train')
            plt.plot(val_precision,label='Val') 
            plt.xlabel('epochs')
            plt.legend()
            plt.subplot(1,2,2)
            plt.title('Recall')
            plt.plot(recall,label='Train')
            plt.plot(val_recall,label='Val') 
            plt.xlabel('epochs')
            plt.legend()
            plt.tight_layout()
            plt.show()
        else:
            pass
    
    
    def confusion_matrix_plot(self):
        if hasattr(self, 'predictions'):
            cm = confusion_matrix(self.Ytest,self.binary_predictions)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm)
            disp.plot()
        else:
            print('Test method should be called before')

    def plot_all_training_metrics(self):
        self.loss_plot()
        self.accuracy_plot()
        self.precision_recall_plot()

    def multiple_seeds_analysis(self,seeds:list):
        pass
          



model 1

In [ ]:
class model_1(DLModel):
    def __init__(self,*args,**kwargs) -> None:
        super().__init__(*args,**kwargs)

    def model_compiler(self):
        MLP_model=keras.Sequential(
                [
                
                    keras.Input(shape=(7,)),
                    keras.layers.Dense(10,activation='tanh', kernel_initializer="glorot_uniform"),
                    keras.layers.Dense(10,activation='tanh'),
                    keras.layers.Dropout(0.4),    
                    keras.layers.Dense(6,activation='tanh'),
                    keras.layers.Dense(6,activation='tanh'),
                    keras.layers.Dense(1,activation='sigmoid')
                ]
                )
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,decay_steps=10000,decay_rate=0.9)
        model_loss=keras.losses.BinaryCrossentropy()
        model_optimizer=keras.optimizers.Adam(learning_rate=lr_schedule)
        model_metrics=[keras.metrics.BinaryAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()]
        MLP_model.compile(loss=model_loss,optimizer=model_optimizer,metrics=model_metrics)
        return MLP_model

    def training_params(self):
        epochs=300
        batch_size=32
        callback = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        return {'callbacks':[callback,tensorboard_callback],'epochs':epochs,'batch_size':batch_size}


    

In [ ]:
model=model_1(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.architecture
model.train(seed=123)
model.plot_all_training_metrics()
model.test()

In [ ]:
model=model_1(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.train(seed=321)
# model.plot_all_training_metrics()
model.test()

model 2

In [ ]:
class model_2(DLModel):
    def __init__(self,*args,**kwargs) -> None:
        super().__init__(*args,**kwargs)
        
    def model_compiler(self):        
        MLP_model=keras.Sequential(
            [
                keras.Input(shape=(7,)),            
                keras.layers.Dense(10,activation='tanh', kernel_initializer="glorot_uniform"),
                keras.layers.BatchNormalization(),
                keras.layers.Dense(10,activation='tanh'),
                keras.layers.Dropout(0.4),    
                keras.layers.Dense(6,activation='tanh'),
                keras.layers.Dense(6,activation='tanh'),
                keras.layers.Dropout(0.7),  
                keras.layers.Dense(10,activation='tanh'),
                keras.layers.Dense(1,activation='sigmoid')
            ]
            )
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,decay_steps=10000,decay_rate=0.9)
        model_loss=keras.losses.BinaryCrossentropy()
        model_optimizer=keras.optimizers.Adam(learning_rate=lr_schedule)
        model_metrics=[keras.metrics.BinaryAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()]
        MLP_model.compile(loss=model_loss,optimizer=model_optimizer,metrics=model_metrics)
        return MLP_model
    
    def training_params(self):
        epochs=300
        batch_size=32
        callback = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        return {'callbacks':[callback,tensorboard_callback],'epochs':epochs,'batch_size':batch_size}



In [ ]:
model=model_2(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.architecture
model.train(seed=123)
model.plot_all_training_metrics()
model.test()

In [ ]:
model=model_2(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.train(seed=123)
model.test()

model 3

In [ ]:
class model_3(DLModel):
    def __init__(self,*args,**kwargs) -> None:
        super().__init__(*args,**kwargs)
        
    def model_compiler(self):        
        MLP_model=keras.Sequential(
            [
                keras.Input(shape=(7,)),            
                keras.layers.Dense(5,activation='tanh', kernel_initializer="glorot_uniform"),
                keras.layers.BatchNormalization(),
                keras.layers.Dense(5,activation='tanh'),
                keras.layers.Dropout(0.4),    
                keras.layers.Dense(3,activation='tanh'),
                keras.layers.Dense(3,activation='tanh'),
                keras.layers.Dropout(0.7),  
                keras.layers.Dense(6,activation='tanh'),
                keras.layers.Dense(4,activation='tanh'),
                keras.layers.Dense(2,activation='tanh'),
                keras.layers.Dense(1,activation='sigmoid')
            ]
            )
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,decay_steps=10000,decay_rate=0.9)
        model_loss=keras.losses.BinaryCrossentropy()
        model_optimizer=keras.optimizers.Adam(learning_rate=lr_schedule)
        model_metrics=[keras.metrics.BinaryAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()]
        MLP_model.compile(loss=model_loss,optimizer=model_optimizer,metrics=model_metrics)

        return MLP_model
    
    def training_params(self):
        epochs=300
        batch_size=32
        callback = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        return {'callbacks':[callback,tensorboard_callback],'epochs':epochs,'batch_size':batch_size}



In [ ]:
model=model_3(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.architecture
model.train(seed=123)
model.plot_all_training_metrics()
model.test()

In [ ]:
model=model_3(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.train(seed=123)
model.test()

model 4

In [ ]:
class model_4(DLModel):
    def __init__(self,*args,**kwargs) -> None:
        super().__init__(*args,**kwargs)
        
    def model_compiler(self):        
        MLP_model=keras.Sequential(
            [
                keras.Input(shape=(7,)),            
                keras.layers.Dense(5,activation='selu', kernel_initializer="glorot_uniform"),
                keras.layers.BatchNormalization(),
                keras.layers.Dense(5,activation='selu'),
                keras.layers.Dropout(0.4),    
                keras.layers.Dense(3,activation='selu'),
                keras.layers.Dense(3,activation='selu'),
                keras.layers.Dropout(0.7),  
                keras.layers.Dense(6,activation='selu'),
                keras.layers.Dense(4,activation='selu'),
                keras.layers.Dense(2,activation='selu'),
                keras.layers.Dense(1,activation='sigmoid')
            ]
            )
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,decay_steps=10000,decay_rate=0.9)
        model_loss=keras.losses.BinaryCrossentropy()
        model_optimizer=keras.optimizers.Adam(learning_rate=lr_schedule)
        model_metrics=[keras.metrics.BinaryAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()]
        MLP_model.compile(loss=model_loss,optimizer=model_optimizer,metrics=model_metrics)

        return MLP_model
    
    def training_params(self):
        epochs=300
        batch_size=32
        callback = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        return {'callbacks':[callback,tensorboard_callback],'epochs':epochs,'batch_size':batch_size}



In [ ]:
model=model_4(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.architecture
model.train(seed=123)
model.plot_all_training_metrics()
model.test()

model 5

In [ ]:
class model_5(DLModel):
    def __init__(self,*args,**kwargs) -> None:
        super().__init__(*args,**kwargs)
        
    def model_compiler(self):        
        MLP_model=keras.Sequential(
            [
                keras.Input(shape=(7,)),            
                keras.layers.Dense(5,activation='tanh', kernel_initializer="glorot_uniform"),
                keras.layers.BatchNormalization(),
                keras.layers.Dense(5,activation='tanh'),
                keras.layers.Dropout(0.4),    
                keras.layers.Dense(3,activation='tanh'),
                keras.layers.Dense(3,activation='tanh'),
                keras.layers.Dropout(0.7),  
                keras.layers.Dense(6,activation='tanh'),
                keras.layers.Dense(4,activation='tanh'),
                keras.layers.Dense(6,activation='tanh'),
                keras.layers.Dense(4,activation='tanh'),
                keras.layers.Dense(6,activation='tanh'),
                keras.layers.Dense(4,activation='tanh'),
                keras.layers.Dense(2,activation='tanh'),
                keras.layers.Dense(1,activation='sigmoid')
            ]
            )
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,decay_steps=10000,decay_rate=0.9)
        model_loss=keras.losses.BinaryCrossentropy()
        model_optimizer=keras.optimizers.Adam(learning_rate=lr_schedule)
        model_metrics=[keras.metrics.BinaryAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()]
        MLP_model.compile(loss=model_loss,optimizer=model_optimizer,metrics=model_metrics)

        return MLP_model
    
    def training_params(self):
        epochs=300
        batch_size=32
        callback = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        return {'callbacks':[callback,tensorboard_callback],'epochs':epochs,'batch_size':batch_size}



In [ ]:
model=model_5(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.architecture
model.train(seed=123)
model.plot_all_training_metrics()
model.test()

model 6

In [ ]:
class model_6(DLModel):
    def __init__(self,*args,**kwargs) -> None:
        super().__init__(*args,**kwargs)
        
    def model_compiler(self):        
        MLP_model=keras.Sequential(
            [
                keras.Input(shape=(7,)),            
                keras.layers.Dense(5,activation='tanh', kernel_initializer="glorot_uniform"),
                keras.layers.Dense(3,activation='tanh'),
                keras.layers.Dense(3,activation='tanh'),
                keras.layers.Dense(1,activation='sigmoid')
            ]
            )
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,decay_steps=10000,decay_rate=0.9)
        model_loss=keras.losses.BinaryCrossentropy()
        model_optimizer=keras.optimizers.Adam(learning_rate=lr_schedule)
        model_metrics=[keras.metrics.BinaryAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()]
        MLP_model.compile(loss=model_loss,optimizer=model_optimizer,metrics=model_metrics)

        return MLP_model
    
    def training_params(self):
        epochs=300
        batch_size=32
        callback = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        return {'callbacks':[callback,tensorboard_callback],'epochs':epochs,'batch_size':batch_size}



In [ ]:
model=model_6(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.architecture
model.train(seed=123)
model.plot_all_training_metrics()
model.test()

In [ ]:
model=model_6(X_train,Y_train,X_val,Y_val,X_test,Y_test)
model.train(seed=321)
model.test()

In [ ]:
%load_ext tensorboard
# %reload_ext tensorboard

CNN

RNN